In [2]:
!pip install yfinance==0.2.38
!pip install pandas==2.2.2
!pip install nbformat

   ---------------------------------------- 0.0/73.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/73.0 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/73.0 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/73.0 kB ? eta -:--:--
   ----------- ---------------------------- 20.5/73.0 kB 165.2 kB/s eta 0:00:01
   ----------- ---------------------------- 20.5/73.0 kB 165.2 kB/s eta 0:00:01
   ---------------- ----------------------- 30.7/73.0 kB 163.8 kB/s eta 0:00:01
   ---------------------- ----------------- 41.0/73.0 kB 164.3 kB/s eta 0:00:01
   ---------------------- ----------------- 41.0/73.0 kB 164.3 kB/s eta 0:00:01
   ---------------------------------------- 73.0/73.0 kB 236.8 kB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.43
    Uninstalling yfinance-0.2.43:
      Successfully uninstalled yfinance-0.2.43
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   --

  You can safely remove it manually.


   --------------- ------------------------ 4.6/11.6 MB 84.2 kB/s eta 0:01:24
   --------------- ------------------------ 4.6/11.6 MB 84.2 kB/s eta 0:01:24
   --------------- ------------------------ 4.6/11.6 MB 84.2 kB/s eta 0:01:24
   --------------- ------------------------ 4.6/11.6 MB 84.2 kB/s eta 0:01:24
   --------------- ------------------------ 4.6/11.6 MB 84.2 kB/s eta 0:01:24
   --------------- ------------------------ 4.6/11.6 MB 84.2 kB/s eta 0:01:24
   --------------- ------------------------ 4.6/11.6 MB 84.2 kB/s eta 0:01:24
   --------------- ------------------------ 4.6/11.6 MB 84.2 kB/s eta 0:01:24
   --------------- ------------------------ 4.6/11.6 MB 84.4 kB/s eta 0:01:24
   --------------- ------------------------ 4.6/11.6 MB 84.4 kB/s eta 0:01:24
   --------------- ------------------------ 4.6/11.6 MB 84.4 kB/s eta 0:01:24
   --------------- ------------------------ 4.6/11.6 MB 84.1 kB/s eta 0:01:24
   --------------- ------------------------ 4.6/11.6 MB 84.1 kB/

In [3]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Question 1: Use yfinance to Extract Stock Data

In [8]:
# Create a Ticker object for Tesla
tesla = yf.Ticker("TSLA")

In [16]:
# Get historical stock price data for the past 5 days
tesla_data = tesla.history(period = "max")
#tesla_data

In [11]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,index,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Question 2: Use Webscraping to Extract Tesla Revenue Data

In [17]:
url = " https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data =  requests.get(url).text

In [20]:
soup = BeautifulSoup(html_data, "html.parser")

In [59]:
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in soup.find("tbody").find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text

    # Finally we append the data of each row to the table
    tesla_revenue = pd.concat([tesla_revenue,pd.DataFrame({"Date":[date], "Revenue":[revenue]})], ignore_index=True) 
#tesla_revenue

In [60]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"", regex=True)
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [26]:
tesla_revenue.tail()

,Date,Revenue
8,2013,2013
9,2012,413
10,2011,204
11,2010,117
12,2009,112


## Question 3: Use yfinance to Extract Stock Data

In [27]:
GameStop = yf.Ticker("GME")

In [29]:
gme_data = GameStop.history(period = "max")
#gme_data

In [30]:
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578048,1.607505,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Question 4: Use Webscraping to Extract GME Revenue Data

In [31]:
url =  "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data_2 =  requests.get(url).text

In [32]:
soup = BeautifulSoup(html_data_2, "html.parser")

In [61]:
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in soup.find("tbody").find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text

    # Finally we append the data of each row to the table
    gme_revenue = pd.concat([gme_revenue,pd.DataFrame({"Date":[date], "Revenue":[revenue]})], ignore_index=True) 
#gme_revenue.head()

In [62]:
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"", regex=True)
#gme_revenue.head()

In [38]:
gme_revenue.dropna(inplace=True)

gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

In [39]:
gme_revenue.tail()

,Date,Revenue
11,2009,8806
12,2008,7094
13,2007,5319
14,2006,3092
15,2005,1843


## Question 5: Plot Tesla Stock Graph

In [53]:
tesla_stock_data = pd.DataFrame(columns=["Date", "Data"])

for row in soup.find("tbody").find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    data = col[1].text

    # Finally we append the data of each row to the table
    tesla_stock_data = pd.concat([tesla_stock_data,pd.DataFrame({"Date":[date], "Data":[data]})], ignore_index=True) 
tesla_stock_data

,Date,Data
0,2020,"$6,466"
1,2019,"$8,285"
2,2018,"$8,547"
3,2017,"$7,965"
4,2016,"$9,364"
5,2015,"$9,296"
6,2014,"$9,040"
7,2013,"$8,887"
8,2012,"$9,551"
9,2011,"$9,474"


In [56]:
tesla_stock_data["Data"] = tesla_stock_data['Data'].str.replace(',|\$',"", regex=True)
tesla_stock_data.dropna(inplace=True)

tesla_stock_data = tesla_stock_data[tesla_stock_data['Data'] != ""]
tesla_stock_data.head()

,Date,Data
0,2020,6466
1,2019,8285
2,2018,8547
3,2017,7965
4,2016,9364


In [64]:
print(tesla_stock_data.columns)

Index(['Date', 'Data'], dtype='object')


In [63]:
make_graph(tesla_stock_data, tesla_revenue, "Tesla Stock Data and Revenue")

AttributeError: 'DataFrame' object has no attribute 'Close'

## Question 6: Plot GameStop Stock Graph

In [67]:
gme_stock_data = pd.DataFrame(columns=["Date", "Data"])

for row in soup.find("tbody").find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    data = col[1].text

    # Finally we append the data of each row to the table
    gme_stock_data = pd.concat([gme_stock_data,pd.DataFrame({"Date":[date], "Data":[data]})], ignore_index=True) 
gme_stock_data.head()

,Date,Data
0,2020,"$6,466"
1,2019,"$8,285"
2,2018,"$8,547"
3,2017,"$7,965"
4,2016,"$9,364"


In [68]:
gme_stock_data["Data"] = gme_stock_data['Data'].str.replace(',|\$',"", regex=True)
gme_stock_data.dropna(inplace=True)

gme_stock_data = gme_stock_data[gme_stock_data['Data'] != ""]
gme_stock_data.head()

,Date,Data
0,2020,6466
1,2019,8285
2,2018,8547
3,2017,7965
4,2016,9364


In [69]:
make_graph(gme_stock_data, gme_revenue, "GME Stock Data and Revenue")

AttributeError: 'DataFrame' object has no attribute 'Close'